<a href="https://colab.research.google.com/github/cesar20c190/CONSULTADEF/blob/main/Acesso_DPB_Tx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [33]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjba/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública

In [36]:
# Aqui trocamos o filtro da classe pelo código do município IBGE
payload = json.dumps({
  "size": 10000,
  "query": {
    "match": {
      "orgaoJulgador.codigoMunicipioIBGE": 2931350
    }
  },
  "sort": [
    {
      "dataAjuizamento": {
        "order": "desc"
      }
    }
  ]
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)
dados_dict = response.json()

# Mostra quantos processos vieram
print(f"Total de processos: {len(dados_dict['hits']['hits'])}")

Total de processos: 10000


In [37]:
display(dados_dict['hits']['hits'][10]) # Exemplo


{'_index': 'api_publica_tjba',
 '_id': 'TJBA_G1_80016990920258050256',
 '_score': None,
 '_source': {'numeroProcesso': '80016990920258050256',
  'classe': {'codigo': 279, 'nome': 'Inquérito Policial'},
  'sistema': {'codigo': 1, 'nome': 'Pje'},
  'formato': {'codigo': 1, 'nome': 'Eletrônico'},
  'tribunal': 'TJBA',
  'dataHoraUltimaAtualizacao': '2025-03-20T12:26:00.102Z',
  'grau': 'G1',
  '@timestamp': '2025-03-21T18:37:20.387277566Z',
  'dataAjuizamento': '2025-03-14T00:00:00.000Z',
  'movimentos': [{'complementosTabelados': [{'codigo': 2,
      'valor': 2,
      'nome': 'sorteio',
      'descricao': 'tipo_de_distribuicao_redistribuicao'}],
    'codigo': 26,
    'nome': 'Distribuição',
    'dataHora': '2025-03-14T00:00:00.000Z'},
   {'codigo': 12281,
    'nome': 'Comunicação eletrônica',
    'dataHora': '2025-03-14T00:00:00.000Z'}],
  'id': 'TJBA_G1_80016990920258050256',
  'nivelSigilo': 0,
  'orgaoJulgador': {'codigoMunicipioIBGE': 2931350,
   'codigo': 76526,
   'nome': '2ª VARA 

In [40]:
if dados_dict['hits']['total']['value'] > 0 and len(dados_dict['hits']['hits']) > 0:
    processo = dados_dict['hits']['hits'][0]['_source']

    numero_processo = processo.get('numeroProcesso', '—')
    classe = processo.get('classe', {}).get('nome', '—')
    data_ajuizamento = processo.get('dataAjuizamento', '—')
    ultima_atualizacao = processo.get('dataHoraUltimaAtualizacao', '—')
    orgao_julgador_data = processo.get('orgaoJulgador', {})
    codigo_orgao_julgador = orgao_julgador_data.get('codigo', '—')
    orgao_julgador = orgao_julgador_data.get('nome', '—')
    Municipio = orgao_julgador_data.get('codigoMunicipioIBGE', '—')

    if processo.get('assuntos') and len(processo['assuntos']) > 0:
        assunto = processo['assuntos'][0].get('nome', '—')
    else:
        assunto = '—'

    print(f'Número do processo: {numero_processo}')
    print(f'Classe: {classe}')
    print(f'Assunto principal: {assunto}')
    print(f'Órgão julgador: {codigo_orgao_julgador} - {orgao_julgador}')
    print(f'Data de ajuizamento: {data_ajuizamento}')
    print(f'Última atualização: {ultima_atualizacao}')
    print(f'Código Município IBGE: {Municipio}')
else:
    print("❌ Nenhum processo encontrado na resposta.")




Número do processo: 00085669120248050256
Classe: Procedimento do Juizado Especial Cível
Assunto principal: Perdas e Danos
Órgão julgador: 17873 - 1ª VARA JUIZADOS ESPECIAIS DE TEIXEIRA DE FREITAS
Data de ajuizamento: 2025-12-16T09:08:36.000Z
Última atualização: 2025-03-20T12:32:07.360Z
Código Município IBGE: 2931350


In [41]:
processos = []

for processo in dados_dict['hits']['hits']:
  numero_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  classe = processo['_source']['classe']['nome']
  assuntos = processo['_source']['assuntos'] # Pode ter mais de um
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  formato = processo['_source']['formato']['nome']
  codigo = processo['_source']['orgaoJulgador']['codigo']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
  sort = processo['sort'][0]
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato, \
                    codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])

df.sample(5)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
5685,00077180720248050256,Termo Circunstanciado,2024-11-05T09:46:49.000Z,2025-03-20T12:29:28.004Z,Eletrônico,17873,1ª VARA JUIZADOS ESPECIAIS DE TEIXEIRA DE FREITAS,2931350,JE,"[{'codigo': 14100, 'nome': 'Raça'}, {'codigo':...","[{'complementosTabelados': [{'codigo': 2, 'val...",1730800009000
5236,80102406520248050256,Procedimento Comum Cível,2024-11-18T00:00:00.000Z,2025-03-19T20:38:16.085Z,Eletrônico,5274,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COME...,2931350,G1,"[{'codigo': 7757, 'nome': 'Auxílio-Doença Acid...","[{'complementosTabelados': [{'codigo': 2, 'val...",1731888000000
2582,00000886020258050256,Execução de Título Extrajudicial,2025-01-14T09:37:55.000Z,2025-03-20T12:27:59.301Z,Eletrônico,18553,2ª VARA JUIZADOS ESPECIAIS DE TEIXEIRA DE FREITAS,2931350,JE,"[{'codigo': 4980, 'nome': 'Nota Promissória'}]","[{'complementosTabelados': [{'codigo': 2, 'val...",1736847475000
2659,80001600820258050256,Reclamação Pré-processual,2025-01-13T00:00:00.000Z,2025-03-19T21:44:47.617Z,Eletrônico,81156,[CEJUSC] TEIXEIRA DE FREITAS,2931350,G1,"[{'codigo': 6239, 'nome': 'Fixação'}]","[{'complementosTabelados': [{'codigo': 2, 'val...",1736726400000
8731,80084418420248050256,Divórcio Consensual,2024-09-04T00:00:00.000Z,2025-01-11T14:19:47.973Z,Eletrônico,5272,1ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COME...,2931350,G1,"[{'codigo': 14923, 'nome': 'Partilha'}]","[{'complementosTabelados': [{'codigo': 2, 'val...",1725408000000


In [47]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([codigo, nome, data_hora])
    return lst_movimentos_final

In [49]:
def gera_lista_movimentos(movimentos):
    lst_movimentos_final = []
    if isinstance(movimentos, list):
        for movimento in movimentos:
            if isinstance(movimento, dict):  # só processa se for dicionário
                codigo = movimento.get('codigo')
                nome = movimento.get('nome')
                data_hora = movimento.get('dataHora')
                lst_movimentos_final.append((codigo, nome, data_hora))
            else:
                # opcional: registrar ou ignorar listas internas ou itens inválidos
                continue
    return lst_movimentos_final


In [52]:
df.to_csv('anpp.csv', sep=',', header=True, index=False)

with open('movimentos_anpp.txt', 'w', encoding='utf-8') as file:
    for index, row in df.iterrows():
        file.write(f"Autos n. {row['numero_processo']}, Classe: {row['classe']}\n")
        file.write(f"Ajuizamento: {row['data_ajuizamento']}\n")
        file.write(f"Assuntos: {row['assuntos']}\n\n")
        for movimento in row['movimentos']:
            file.write(f"{str(movimento[2])} | Cód: {str(movimento[0])} | Mov.: {movimento[1]}\n")
        file.write('\n---------------------------------------------------\n\n')
